[Reference](https://medium.com/intel-analytics-software/scalable-autoxgboost-using-analytics-zoo-automl-30d576cb138a)

In [ ]:
pip install analytics-zoo

     |████████████████████████████████| 206.9 MB 58 kB/s 
     |████████████████████████████████| 114.0 MB 67 kB/s 
     |████████████████████████████████| 218.4 MB 66 kB/s 
     |████████████████████████████████| 197 kB 32.8 MB/s 
  Created wheel for pyspark: filename=pyspark-2.4.6-py2.py3-none-any.whl size=218814407 sha256=c3d6f9469266b51c91486ba8db0d1a82ee2a2b01f559b887c8dad487554ebc15
  Stored in directory: /root/.cache/pip/wheels/f1/42/b0/ba397759613f4feb1611021a2503e60e344e546671b2ae04f8
Successfully built pyspark


In [ ]:
pip install ray[default]

     |████████████████████████████████| 128 kB 4.3 MB/s 
     |████████████████████████████████| 192 kB 40.4 MB/s 
     |████████████████████████████████| 201 kB 37.3 MB/s 
     |████████████████████████████████| 3.0 MB 38.9 MB/s 
     |████████████████████████████████| 65 kB 3.3 MB/s 
     |████████████████████████████████| 82 kB 212 kB/s 
     |████████████████████████████████| 1.1 MB 47.6 MB/s 
     |████████████████████████████████| 160 kB 49.5 MB/s 
     |████████████████████████████████| 271 kB 58.6 MB/s 
     |████████████████████████████████| 85 kB 3.5 MB/s 
     |████████████████████████████████| 57 kB 4.4 MB/s 
  Created wheel for gpustat: filename=gpustat-1.0.0b1-py3-none-any.whl size=15979 sha256=9f7cc2242b0200575a1999e8e1e16f76af9d0d5bedd00662362a05f2ad41ff51
  Stored in directory: /root/.cache/pip/wheels/1a/16/e2/3e2437fba4c4b6a97a97bd96fce5d14e66cff5c4966fb1cc8c
Successfully built gpustat


# Prepare Context


In [ ]:
from zoo.orca import init_orca_context, stop_orca_context
init_orca_context(cluster_mode="local", cores=112, memory='20g',
                  init_ray_on_spark=True)

Initializing orca context
Current pyspark location is : /usr/local/lib/python3.7/dist-packages/pyspark/__init__.py
Start to getOrCreate SparkContext
pyspark_submit_args is:  --driver-class-path /usr/local/lib/python3.7/dist-packages/zoo/share/lib/analytics-zoo-bigdl_0.13.0-spark_2.4.6-0.11.1-jar-with-dependencies.jar:/usr/local/lib/python3.7/dist-packages/bigdl/share/lib/bigdl-0.13.0-jar-with-dependencies.jar --driver-class-path /usr/local/lib/python3.7/dist-packages/zoo/share/lib/analytics-zoo-bigdl_0.13.0-spark_2.4.6-0.11.1-jar-with-dependencies.jar:/usr/local/lib/python3.7/dist-packages/bigdl/share/lib/bigdl-0.13.0-jar-with-dependencies.jar --driver-class-path /usr/local/lib/python3.7/dist-packages/zoo/share/lib/analytics-zoo-bigdl_0.13.0-spark_2.4.6-0.11.1-jar-with-dependencies.jar:/usr/local/lib/python3.7/dist-packages/bigdl/share/lib/bigdl-0.13.0-jar-with-dependencies.jar pyspark-shell 
Successfully got a SparkContext


2022-01-13 23:56:57,853	INFO services.py:1340 -- View the Ray dashboard at http://172.28.0.2:8265


{'node_ip_address': '172.28.0.2', 'raylet_ip_address': '172.28.0.2', 'redis_address': '172.28.0.2:50826', 'object_store_address': '/tmp/ray/session_2022-01-13_23-56-54_553547_61/sockets/plasma_store', 'raylet_socket_name': '/tmp/ray/session_2022-01-13_23-56-54_553547_61/sockets/raylet', 'webui_url': '172.28.0.2:8265', 'session_dir': '/tmp/ray/session_2022-01-13_23-56-54_553547_61', 'metrics_export_port': 57001, 'node_id': '0fca83e7a6f051212c4eb607c856458ff84f6ab0f41f8c2b1508700a'}


<SparkContext master=local[112] appName=pyspark-shell>

# Create an AutoXGBoost Classifier


In [ ]:
config = {"tree_method":'hist', "learning_rate":0.1, "gamma":0.1,
          "min_child_weight":30, "reg_lambda":1,
          "scale_pos_weight":2, "subsample":1, "n_jobs":56}
Auto_xgb_clf = AutoXGBClassifier(cpus_per_trial=4,
                                 name="auto_xgb_classifier",
                                 **config)

# Train the AutoXGBoost Classifier


In [ ]:
search_space = {"n_estimators": hp.grid_search([50, 1000]),
                "max_depth": hp.randint(2, 15)}
auto_xgb_clf.fit(data=(X_train, y_train),
                 validation_data=(X_val, y_val),
                 metric="error",
                 metric_mode="min",
                 n_sampling=2,
                 search_space=search_space)

# Retrieve the Best Model from AutoXGBoost

In [ ]:
best_model = auto_xgb_clf.get_best_model()
accuracy = best_model.evaluate(X_val, y_val, metrics=["accuracy"])